# js

In [ ]:
import json
import os
import re
import shutil
from pathlib import Path

import doit
import importnb
import yaml

with importnb.Notebook():
    from jupyak.tasks import _actions as A
    from jupyak.tasks import _well_known as W
    from jupyak.tasks import _yak as Y

In [ ]:
DEP_YML_PATTERN = "build/npm-dist/{}/packages.yml"

In [ ]:
def js_hack_tasks(yak: Y.Yak):
    yield dict(
        name="not-a-package",
        doc="> create an empty project which can be linked against to avoid downloading/solving",
        targets=[yak.not_a_package_json],
        actions=[(_write_not_a_package, [yak.not_a_package_json])],
    )
    yield dict(
        name="yarnrc",
        uptodate=[doit.tools.config_changed(YARNRC_TMPL)],
        targets=[yak.yarnrc_path],
        actions=[(_write_yarnrc, [yak])],
    )

In [ ]:
YARNRC_TMPL = """# some additional per-project values are set via environment variable
enableImmutableInstalls: false
enableInlineBuilds: false
enableTelemetry: false
httpTimeout: 60000
nodeLinker: node-modules
npmRegistryServer: https://registry.npmjs.org/
globalFolder: {cache_path}
cacheFolder: {cache_path}

# these messages provide no actionable information, and make non-TTY output
# almost unreadable, masking real dependency-related information
# see: https://yarnpkg.com/advanced/error-codes
logFilters:
  - code: YN0072 # PORTALS
    level: discard
  - code: YN0057 # RESOLUTIONS_IGNORED
    level: discard
  - code: YN0002 # DOESNT_PROVIDE
    level: discard
  - code: YN0060 # DOESNT_PROVIDE_VERSION
    level: discard
  - code: YN0006 # SOFT_LINK_BUILD
    level: discard
  - code: YN0007 # MUST_BUILD
    level: discard
  - code: YN0008 # MUST_REBUILD
    level: discard
  - code: YN0013 # FETCH_NOT_CACHED
    level: discard
  - code: YN0019 # UNUSED_CACHE_ENTRY
    level: discard
"""

In [ ]:
def _write_yarnrc(yak: Y.Yak):
    yak.yarnrc_path.parent.mkdir(parents=True, exist_ok=True)
    yak.yarnrc_path.write_text(
        YARNRC_TMPL.format(cache_path=yak.yarn_cache_path), encoding="utf-8"
    )

In [ ]:
def _write_not_a_package(package_json: Path):
    package_json.parent.mkdir(parents=True, exist_ok=True)
    package_json.write_text(
        json.dumps(
            {
                "name": "not-a-package",
                "description": "not-a-package for painful resolutions",
                "version": "0.0.0",
            },
            indent=2,
            sort_keys=True,
        ),
        encoding="utf-8",
    )

In [ ]:
def js_repo_tasks(repo: Y.Repo):
    yak = repo.parent
    js = repo.js
    package_jsons = js.package_jsons
    work_path, in_repo = repo.run_context
    install_deps = [yak.env.venv_history, yak.yarnrc_path, *package_jsons]

    pre_install_actions = []

    install_deps += [yak.not_a_package_json]

    link_deps = [yak.work_path / DEP_YML_PATTERN.format(dep) for dep in js.dependencies]
    install_deps += link_deps
    pre_install_actions = [
        A.git(["reset", "--hard", "HEAD"], in_repo),
        (
            _fix_js_resolutions,
            [
                js.root_package_json,
                link_deps,
                yak.not_a_package_json,
                js.all_install_exclude_resolutions,
                js.link_exclude_patterns,
            ],
        ),
    ]

    yield dict(
        name=f"{repo.name}:yarn:install",
        doc=f"> install npm dependencies of {repo.name}",
        actions=[*pre_install_actions, A.run([*yak.env.run_args, "yarn"], in_repo)],
        file_dep=install_deps,
        targets=[js.yarn_state],
    )

    dist_deps = []
    for path, task_dicts in js.tasks.items():
        for task_dict in task_dicts:
            for task in yarn_task(work_path / path, task_dict, repo):
                dist_deps += task["targets"]
                yield task

    dist_yml = yak.work_path / DEP_YML_PATTERN.format(repo.name)

    yield dict(
        name=f"{repo.name}:dist",
        uptodate=[doit.tools.config_changed({"exclude": js.dist_exclude_patterns})],
        actions=[(_js_dist, [dist_yml, repo])],
        file_dep=[*package_jsons, *dist_deps],
        targets=[dist_yml],
    )

> run an `npm` command

In [ ]:
def yarn_task(task_path: Path, task: dict, repo: Y.Repo):
    work_path, in_repo = repo.run_context
    in_repo["cwd"] = task_path
    actions = task["actions"]
    file_dep = A.resolve_globbish(repo.work_path, task["file_dep"])
    targets = A.resolve_globbish(repo.work_path, task["targets"])
    file_dep += [
        repo.parent.env.py_site_packages / _pth_path(name)
        for name in task.get("needs_pth", [])
    ]
    name = task["name"]
    path = "." if task_path == repo.work_path else task_path.name
    yield dict(
        name=f"{repo.name}:yarn:{name}:{path}",
        doc=f"> ensure {repo.name} {name} in {path}",
        actions=[
            A.run([*repo.parent.env.run_args, *action], in_repo) for action in actions
        ],
        file_dep=[repo.js.yarn_lock, repo.js.yarn_state, *file_dep],
        targets=targets,
    )

In [ ]:
def _fix_js_resolutions(
    package_json: Path,
    tgz_lists: list[Path],
    not_a_package_json: Path,
    install_exclude_resolutions: list[str],
    link_exclude_patterns: list[str],
):
    print(f"   ...  fixing resolutions for {package_json.parent.name}")
    pkg_data = json.loads(package_json.read_text(encoding="utf-8"))
    dep_groups = ["dependencies", "devDependencies", "resolutions"]
    rel_not_a_package = (
        f"""link:{os.path.relpath(not_a_package_json.parent, package_json.parent)}"""
    )

    for tgz_list in tgz_lists:
        resolutions = yaml.safe_load(tgz_list.read_text(encoding="utf-8"))
        dest = package_json.parent.name
        src = tgz_list.parent.name
        print(f"   ... {dest} will use {src}")
        for pkg_name, info in resolutions.items():
            if any(re.search(p, pkg_name) for p in link_exclude_patterns):
                print(f"     ...  {pkg_name} is excluded")
                continue
            rel = os.path.relpath(info["path"], package_json.parent)
            link = f"""file:{rel}"""
            for dep_group in dep_groups:
                if dep_group not in pkg_data:
                    if dep_group != "resolutions":
                        continue
                    pkg_data["resolutions"] = {}
                if dep_group == "resolutions" or pkg_name in pkg_data[dep_group]:
                    pkg_data[dep_group][pkg_name] = link

    for pkg_name in install_exclude_resolutions:
        print(f"     ...  {pkg_name} is excluded")
        pkg_data.setdefault("resolutions", {}).update({pkg_name: rel_not_a_package})

    package_json.write_text(json.dumps(pkg_data, indent=2))

    for child_package_json in _find_workspace_packages(pkg_data, package_json):
        _fix_js_resolutions(
            child_package_json,
            tgz_lists,
            not_a_package_json,
            install_exclude_resolutions,
            link_exclude_patterns,
        )

In [ ]:
def _find_workspace_packages(pkg_data: dict, pkg_json: Path):
    workspaces = pkg_data.get("workspaces")
    if not workspaces:
        return []
    return sorted(
        set(
            sum(
                [
                    sorted(pkg_json.parent.glob(f"{ws_glob}/package.json"))
                    for ws_glob in (
                        workspaces["packages"]
                        if "packages" in workspaces
                        else workspaces
                    )
                ],
                [],
            ),
        ),
    )

In [ ]:
def _js_dist(tgz_list: Path, repo: Y.Repo):
    exclude_patterns = repo.js.dist_exclude_patterns or []
    dist = tgz_list.parent
    in_dist = {"cwd": dist}
    package_json = repo.work_path / W.PACKAGE_JSON
    pkg = json.loads(package_json.read_text(encoding="utf-8"))
    package_jsons = _find_workspace_packages(pkg, package_json)
    if not package_jsons:
        raise RuntimeError(f"{work_dir.name} has no workspaces")
    if dist.is_dir():
        shutil.rmtree(dist)
    dist.mkdir(exist_ok=True, parents=True)
    tgz_info = {}
    for package_json in sorted(package_jsons):
        if any(
            xp in str(package_json.relative_to(repo.parent.work_path))
            for xp in exclude_patterns
        ):
            print(
                f"   ... skipping {package_json.parent.relative_to(repo.work_path)} by exclude pattern",
            )
            continue
        pkg_name = json.loads(package_json.read_text(encoding="utf-8"))["name"]
        tgz_info[pkg_name] = {"path": str(package_json.parent)}
    tgz_list.write_text(yaml.safe_dump(tgz_info), encoding="utf-8")

In [ ]:
def _pth_path(name: str):
    return f"""_{name.replace("-", "_")}.pth"""